# **ИМПОРТИРОВАТЬ ВСЕ БИБЛИОТЕКИ**

---



In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier
import pickle
!pip install scipy==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **ЗАГРУЗКА ДАННЫХ**

---



In [ ]:
df = pd.read_csv('vehicles_dataset.csv')
df.head(3)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,type,paint_color,image_url,description,county,state,lat,long,posting_date,price_category
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,good,NaN,...,pickup,silver,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,NaN,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,6 cylinders,...,SUV,black,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,NaN,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,good,NaN,...,sedan,NaN,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,NaN,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high


# **ПОДГОТОВКА ДАННЫХ**

---



In [ ]:
#оценим размер выборки
print('Размер выбоpки: {}, {}'.format(df.shape[0], df.shape[1]))

Размер выбоpки: 10050, 27


In [ ]:
#оценим типы данных  в выборке
print('Информация о выборке:\n')
df.info()

Информация о выборке:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10050 entries, 0 to 10049
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              10050 non-null  int64  
 1   url             10050 non-null  object 
 2   region          10050 non-null  object 
 3   region_url      10050 non-null  object 
 4   price           10050 non-null  int64  
 5   year            10014 non-null  float64
 6   manufacturer    9665 non-null   object 
 7   model           9921 non-null   object 
 8   condition       6268 non-null   object 
 9   cylinders       5953 non-null   object 
 10  fuel            9987 non-null   object 
 11  odometer        10007 non-null  float64
 12  title_status    9884 non-null   object 
 13  transmission    10005 non-null  object 
 14  VIN             6086 non-null   object 
 15  drive           7026 non-null   object 
 16  size            2825 non-null   object 
 17  type    

In [ ]:
# оценим дублирование
df.duplicated()
df[df.duplicated()].shape
# есть 50 повторяющихся строчек. удалим их
df2 = df.drop_duplicates()
df2[df2.duplicated()].shape
# дубликатов нет

(0, 27)

In [ ]:
# оценим пропуски
missing_values = ((df2.isna().sum() / len(df2)) * 100).sort_values()
print('Процент пропущенных значений до удаления столбцов:', missing_values)

# удалим колонки с пропусками больше 20% county, cylinders, VIN, type, paint_color, drive, condition, size                                                                                 
df3 = df2.drop(['county', 'cylinders', 'VIN', 'type', 'paint_color', 'drive', 'condition', 'size'], axis=1)
df3.shape
# нужно заменить или удалить пропуски, составляющие меньше 20%
missing_values = ((df3.isna().sum() / len(df3)) * 100).sort_values()
print('Процент пропущенных значений после удаления столбцов:', missing_values)
print('Размер выбоpки: {}, {}'.format(df3.shape[0], df3.shape[1]))


Процент пропущенных значений до удаления столбцов: id                  0.00
state               0.00
price               0.00
price_category      0.00
region              0.00
url                 0.00
region_url          0.00
description         0.02
image_url           0.02
posting_date        0.02
year                0.36
odometer            0.43
transmission        0.45
fuel                0.63
lat                 0.98
long                0.98
model               1.28
title_status        1.66
manufacturer        3.82
type               21.94
paint_color        29.50
drive              30.06
condition          37.66
VIN                39.38
cylinders          40.80
size               71.91
county            100.00
dtype: float64
Процент пропущенных значений после удаления столбцов: id                0.00
state             0.00
price             0.00
price_category    0.00
region            0.00
url               0.00
region_url        0.00
posting_date      0.02
image_url         0.0

In [ ]:
# пропуск по производителю manufacturer, топ значений ford - заполним по нему
df3['manufacturer'].describe()
df3.manufacturer = df3.manufacturer.fillna('ford')
print("Количество пропущенных значений в manufacturer:", df3.manufacturer.isna().sum())

# пропуск по производителю title-status, топ значений clean - заполним по нему
df3['title_status'].describe()
df3.title_status = df3.title_status.fillna('clean')
print("Количество пропущенных значений в title_status:", df3['title_status'].isna().sum())

# пропуск по производителю model, топ значений f-150 - заполним по нему
df3['model'].describe()
df3.model = df3.model.fillna('f-150')
print("Количество пропущенных значений в model:", df3['model'].isna().sum())

# пропуск по производителю long & lat удалим
df3 = df3[df3.lat.notna() | df3.long.notna()]

# пропуск по году производства
df3 = df3[df3.year.notna()]
df3.year = df3.year.astype('int64')

# пропуск по одометру
df3 = df3[df3.odometer.notna()]

# пропуск по transmission
df3 = df3[df3.transmission.notna()]

# пропуск по fuel
df3 = df3[df3.fuel.notna()]
df3.fuel.astype('category')

missing_values = ((df3.isna().sum() / len(df3)) * 100).sort_values()
print(missing_values)

print(df3.info())

Количество пропущенных значений в manufacturer: 0
Количество пропущенных значений в title_status: 0
Количество пропущенных значений в model: 0
id                0.0
long              0.0
lat               0.0
state             0.0
description       0.0
image_url         0.0
transmission      0.0
title_status      0.0
posting_date      0.0
odometer          0.0
model             0.0
manufacturer      0.0
year              0.0
price             0.0
region_url        0.0
region            0.0
url               0.0
fuel              0.0
price_category    0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9742 entries, 0 to 10049
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9742 non-null   int64  
 1   url             9742 non-null   object 
 2   region          9742 non-null   object 
 3   region_url      9742 non-null   object 
 4   price           9742 non-null   int64  
 

In [ ]:
#выбросы в годе выпуска
df3.year.describe()
def calculate_outliers(data):
    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)
    
    return boundaries

boundaries = calculate_outliers(df3.year)
is_outlier = (df3.year < boundaries[0]) | (df3.year > boundaries[1]) 

print('границы значений года', boundaries)
print('кол-во выбросов в значении года',is_outlier.sum())

df3.loc[is_outlier, 'year'] = int(boundaries[0]) + 1

print('кол-во выбросов в значении года после заполнения нижней границей',is_outlier.sum())
print(df3.year.describe())

# ранее минимальный год выпуска был 1915, теперь 1995

границы значений года (1994.5, 2030.5)
кол-во выбросов в значении года 393
кол-во выбросов в значении года после заполнения нижней границей 393
count    9742.000000
mean     2011.684048
std         6.438973
min      1995.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64


In [ ]:
#выбросы в цене авто
df3.price.describe()
def calculate_outliers(data):
    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)
    
    return boundaries

boundaries = calculate_outliers(df3.price)
is_outlier = (df3.price < boundaries[0]) | (df3.price > boundaries[1]) 
df3[is_outlier]
# выбросы только среди максимальных значений. поэтому заменим их
df3.loc[is_outlier, 'price'] = int(boundaries[1])

print('границы значениях цены', boundaries)
print('кол-во выбросов в значении цены',is_outlier.sum())
print(df3.price.describe())

границы значениях цены (-22485.0, 58275.0)
кол-во выбросов в значении цены 205
count     9742.000000
mean     18906.389140
std      13696.175497
min        500.000000
25%       7800.000000
50%      15590.000000
75%      27990.000000
max      58275.000000
Name: price, dtype: float64


In [ ]:
# тип данных дата выпуска автомобиля. заменим o на дату
pd.to_datetime(df3.posting_date)
pd.to_datetime(df3.posting_date, utc=True)
df3['date'] = pd.to_datetime(df3.posting_date, utc=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9742 entries, 0 to 10049
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              9742 non-null   int64              
 1   url             9742 non-null   object             
 2   region          9742 non-null   object             
 3   region_url      9742 non-null   object             
 4   price           9742 non-null   int64              
 5   year            9742 non-null   int64              
 6   manufacturer    9742 non-null   object             
 7   model           9742 non-null   object             
 8   fuel            9742 non-null   object             
 9   odometer        9742 non-null   float64            
 10  title_status    9742 non-null   object             
 11  transmission    9742 non-null   object             
 12  image_url       9742 non-null   object             
 13  description     9742 non-null   

## *Количественные*

In [ ]:
df3[['id', 'price', 'year', 'odometer', 'lat', 'long']].describe()

,id,price,year,odometer,lat,long
count,9.742000e+03,9742.000000,9742.000000,9.742000e+03,9742.000000,9742.000000
mean,7.311557e+09,18906.389140,2011.684048,9.605809e+04,38.610721,-94.081774
std,4.367591e+06,13696.175497,6.438973,8.718621e+04,5.841340,18.066808
min,7.301586e+09,500.000000,1995.000000,0.000000e+00,-67.144243,-158.069300
25%,7.308181e+09,7800.000000,2008.000000,3.914375e+04,34.846764,-108.572925
50%,7.312781e+09,15590.000000,2013.000000,8.878350e+04,39.300000,-87.979200
75%,7.315288e+09,27990.000000,2017.000000,1.379675e+05,42.440000,-80.830000
max,7.317090e+09,58275.000000,2022.000000,3.245000e+06,64.947500,173.885502


## *Категориальные*

In [ ]:
df3[['url', 'region', 'region_url', 'manufacturer', 'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'price_category', 'posting_date']].describe()

,url,region,region_url,manufacturer,model,fuel,title_status,transmission,image_url,description,state,price_category,posting_date
count,9742,9742,9742,9742,9742,9742,9742,9742,9742,9742,9742,9742,9742
unique,9742,393,405,39,3389,5,6,3,8842,9592,51,3,9703
top,https://chattanooga.craigslist.org/ctd/d/chatt...,"kansas city, MO",https://kansascity.craigslist.org,ford,f-150,gas,clean,automatic,https://images.craigslist.org/00N0N_1xMPvfxRAI...,PLEASE VIEW THE FREE CARFAX HISTORY REPORT AT ...,ca,high,2021-04-18T15:40:37-0400
freq,1,86,86,1945,270,8233,9430,7501,188,7,1086,3396,2


# **Feature Generation**

---
Существуют различные способы повышения качества работы моделей машинного обучения. Одним из таких способов является создание новых признаков:

1) Математическое преобразование
2) Групповое преобразование
3) Счет количества признаков


In [ ]:
# для чего нужна генерация признаков?
# Снижение скорости расчета или требуемого объема данных
# Улучшение интерпретируемость модели
# Повышение точности предсказаний
# Как правильно сгенерировать признаки. от чего мы оттакливаемся?

df3['model_len'] = df3.apply(lambda x: len(x.model) , axis=1)
df3['model_len']

df3['age_category'] = df3.apply(lambda x: 'new' if x.year > 2013 else ('old' if x.year < 2006 else 'average'), axis = 1)
df3['age_category']

df3['model_word_count'] = df3.apply(lambda x: len(x.model.split(' ')), axis=1)
df3['model_word_count']

df3['short_model'] = df3.model.apply(lambda x: x.lower().split(' ')[0])
df3['short_model']


0            2500
2            golf
3            rav4
4          altima
5               1
           ...   
10045        rav4
10046    wrangler
10047          a3
10048     cayenne
10049        1500
Name: short_model, Length: 9742, dtype: object

## OHE Кодирование

In [ ]:
data = df3[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category', 'fuel']]
data
ohe = OneHotEncoder(sparse=False)
ohe
ohe.fit(df3[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category', 'fuel']])
ohe.get_feature_names()
ohe_categor = ohe.transform(df3[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category', 'fuel']])
ohe_categor.shape
df3[ohe.get_feature_names()] = ohe_categor
df3.shape

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(9742, 1464)


# **НОРМАЛИЗАЦИЯ, СТАБИЛИЗАЦИЯ ДАННЫХ**


---




In [ ]:
# нормализация нужна, чтобы привезти данные к нормальному виду [0:1]
#чтобы переменная не могла влиять на другую больше чем остальные
# Cтандартизация или Z масштабирование

std_scaler = StandardScaler()
std_scaler
std_scaler.fit(df3[['odometer', 'price', 'lat', 'long', 'year', 'model_len', 'model_word_count', 'model_len', 'model_word_count']])
print('Среднее:', std_scaler.mean_)
print('Дисперсия', std_scaler.var_)
std_scaled = std_scaler.transform(df3[['odometer', 'price', 'lat', 'long', 'year', 'model_len', 'model_word_count', 'model_len', 'model_word_count']])
std_scaled

# запишем новые значение стандартизированных данных
df3[['std_scaled_odometer', 'std_scaled_price', 'std_scaled_lat', 'std_scaled_long', 'std_scaled_year', 'std_scaled_model_len', 'std_scaled_model_word_count', 'std_scaled_model_len', 'std_scaled_model_word_count']] = std_scaled


Среднее: [ 9.60580893e+04  1.89063891e+04  3.86107208e+01 -9.40817742e+01
  2.01168405e+03  1.19616095e+01  2.17060152e+00  1.19616095e+01
  2.17060152e+00]
Дисперсия [7.60065506e+09 1.87565968e+08 3.41177559e+01 3.26376041e+02
 4.14561180e+01 7.50443073e+01 2.17003288e+00 7.50443073e+01
 2.17003288e+00]


In [ ]:

# еще одна нормализация - это min-max нормализация значения [0;1]
from sklearn.preprocessing import MinMaxScaler
df3[['lat', 'long']]
mm_scaler = MinMaxScaler()
mm_scaler
mm_scaler.fit(df3[['lat', 'long']])
mm_scaled = mm_scaler.transform(df3[['lat', 'long']])
mm_scaled
df3[['lat_mm', 'long_mm']] = mm_scaled
data = df3[['lat', 'std_scaled_lat', 'lat_mm', 'long', 'std_scaled_long', 'long_mm']]
data
df3[['lat', 'std_scaled_lat', 'lat_mm', 'long', 'std_scaled_long', 'long_mm']].describe()

# Матрица корреляций для всех географических признаков, чтобы убедиться, что нормализация признаков сохранила свойства исходных признаков.
df3[['lat', 'std_scaled_lat', 'lat_mm', 'long', 'std_scaled_long', 'long_mm']].corr()

,lat,std_scaled_lat,lat_mm,long,std_scaled_long,long_mm
lat,1.000000,1.000000,1.000000,-0.167957,-0.167957,-0.167957
std_scaled_lat,1.000000,1.000000,1.000000,-0.167957,-0.167957,-0.167957
lat_mm,1.000000,1.000000,1.000000,-0.167957,-0.167957,-0.167957
long,-0.167957,-0.167957,-0.167957,1.000000,1.000000,1.000000
std_scaled_long,-0.167957,-0.167957,-0.167957,1.000000,1.000000,1.000000
long_mm,-0.167957,-0.167957,-0.167957,1.000000,1.000000,1.000000


In [ ]:
# добавим признак месяц выставки объявления о продаже авто
df3['month'] = df3['date'].dt.month
df3['dayofweek'] = df3['date'].apply(lambda dw: pd.to_datetime(dw).weekday())
df3['year_postadv'] = df3['date'].dt.year
df3['diff_years'] = df3['year_postadv'] - df3['year']  
df3[['posting_date', 'date', 'month', 'dayofweek', 'diff_years', 'year_postadv']]


# Применим стандартизацию к новым признаками и сохраните результат в новые признаки вида month_std.
std_scaler = StandardScaler()
std_scaler

std_scaler.fit(df3[['month', 'diff_years', 'dayofweek']])
std_scaled = std_scaler.transform(df3[['month', 'diff_years', 'dayofweek']])
std_scaled
df3[['month_std', 'dayofweek_std', 'diff_years_std']] = std_scaled
df3[['month_std', 'dayofweek_std', 'diff_years_std', 'month', 'diff_years', 'dayofweek']]

,month_std,dayofweek_std,diff_years_std,month,diff_years,dayofweek
0,-0.615874,-1.291569,1.117509,4,1,5
2,-0.615874,-0.825632,-0.377037,4,4,2
3,-0.615874,-0.204383,1.117509,4,8,5
4,1.623709,-1.446881,-1.373401,5,0,0
5,-0.615874,-0.049071,0.121145,4,9,3
...,...,...,...,...,...,...
10045,-0.615874,1.504051,1.117509,4,19,5
10046,1.623709,0.572178,-1.373401,5,13,0
10047,-0.615874,0.106241,0.121145,4,10,3
10048,-0.615874,-0.515008,-0.377037,4,6,2


# **ЛИНЕЙНАЯ РЕГРЕССИЯ** 


---



In [ ]:
df_prepared = df3.drop(columns=['id', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date',
                    'age_category', 'short_model', 'price_category', 'date'])


LinRegr = df_prepared.copy()
LinRegr.shape

(9742, 1447)

In [ ]:
# Подготовка данных.
#Разделите выборку на треин и тест в отношении 70/30
train, test = train_test_split(LinRegr, test_size=0.3, random_state=42)
print('выборка всего датасет =', len(LinRegr), '-', 100, '%')
print('тренировочная выборка =', len(train), '-', len(train)*100/len(LinRegr), '%')
print('тестова выборка =', len(test), '-', len(test)*100/len(LinRegr), '%')

выборка всего датасет = 9742 - 100 %
тренировочная выборка = 6819 - 69.99589406692671 %
тестова выборка = 2923 - 30.00410593307329 %


In [ ]:
train_noprice = train.drop(columns='price')
test_noprice = test.drop(columns='price')
#Обучите модель линейной регрессии с дефолтными параметрами.
linregr = LinearRegression()
linregr.fit(train_noprice, train.price)

LinearRegression()

In [ ]:
#Ошибка при расчете линейрной регрессии mae на тестовой выборке составляет 0,001 $.
from sklearn.metrics import mean_absolute_error
pred = linregr.predict(test_noprice)
mean_absolute_error(test.price, pred)

2.34714430276589e-06

# **ВЫВОДЫ ПО ЛИНЕЙНОЙ РЕГРЕССИИ**


---


Средние значения стоимости авто по проивзодителям от 11692 до 22283 $

Погрешность при расчете линейной регрессии 0,001, очень малая погрешность


---



# **ПЕРЕД НАЧАЛОМ ЛЮБОГО МОДЕЛИРОВАНИЯ**
1. ОПРЕДЕЛИТЬ МЕТРИКУ
ОПРЕДЕЛИТЬ ЦЕЛЕВУЮ ПЕРЕМЕННУЮ И ФИЧИ
2. ВЫБРАТЬ АЛГОРИТМ
3. ОБУЧИТЬ МОДЕЛЬ
4. ОЦЕНИТЬ КАЧЕСТВО МОДЕЛИЭ

ЕСТЬ ОБУЧЕНИЕ С УЧИТЕЛЕМ (НА РАЗМЕЧЕННОЙ ВЫБОРКЕ) ПРОНУМЕРОВАНА ЗНАЧЕНИЯМИ ЦЕЛЕВЫХ ПЕРЕМЕННЫХ) ТЕ ЕСТЬ СТОЛБЕЦ 0 1
ОБУЧЕНИЕ БЕЗ УЧИТЕЛЯ (ДАННЫЕ НЕ РАЗМЕЧЕНЫ ЗНАЧЕНИЯМИ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ)

ТРИ ОСНОВНЫХ ТИПА МАШИННОГО ОБУЧЕНИЯ:


---


КЛАССИФИКАЦИЯ - КАТЕГОРИАЛЬНАЯ ВЕЛИЧИНА (ПОНРАВИТСЯ ИЛИ НЕТ КНИГА 0 - 1 БИНАРНАЯ, ИЛИ КОЛ-ВО МАШИН НА ФОТО - МНОГОКЛАССОВАЯ)



---


РЕГРЕССИЯ - ПРЕДСКАЗАТЬ НЕПРЕРЫВНУЮ ВЕЛИЧИНУ (СТОИМОСТЬ КВ ПО ЕЕ ХАР-КИ)


---



КЛАСТЕРИЗАЦИЯ - ПРЕДСКАЗАТЬ ГРУППУ/КЛАСТЕР (ВНУТРИ ГРУППЫ/КЛАСТЕРА ОБЪЕКТЫ ПОХОЖИ ДРУГ НА ДРУГА И НЕ ПОХОЖИ НА ОБЪЕКТЫ ИЗ ДРУГИХ ГРУПП) ДОПУСТИМ  РАЗБИТЬ КЛИЕНТОВ ПО ИХ ПОКУПАТЕЛЬСКОЙ СПОСОБНОСТИ


---



КАК ОПРЕДЕЛИТЬ КАЧ-ВО МОДЕЛИ - ПОСЧИТАТЬ МЕТРИКУ
КАК ПОСЧИТАТЬ МЕТРИКУ? - СТАНДАРТНЫЙ СПОСОБ TRAIN TEST SPLIT
НА ТРЕНИРОВОЧНОЙ ОБУЧАЮТ МОДЕЛЬ, НА ТЕСТОВОЙ ЗАМЕРЯЮТ ЕЕ КАЧ-ВО
пропорции 80/20 или 70/30 , где большая часть - тренировочная



---
МЕТРИКА КАЧЕСТВА КЛАССИФИКАЦИИ - ACCURACY
СКОЛЬКО ПРОЦЕНТОВ ЗНАЧЕНИЙ БЫЛО УГАДАНО
ОТНОШЕНИЕ ПРАВИЛЬНЫХ ПРОГНОЗОВ/К ОБЩЕМУ КОЛ-ВУ

---

МАТРИЦА ОШИБОК CONFUSION MATRIX


# **ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ** 


---



In [ ]:
df_prepared = df3.drop(columns=['id', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date',
                    'age_category', 'short_model', 'date', 'price'])

LogRegr = df_prepared.copy()
LogRegr.shape

# подготовим данные для train/test split
x = LogRegr.drop(['price_category'], axis=1)
y = LogRegr.price_category
# инициализация логист регресии с дефолт параметрами
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print('выборка всего датасет =', len(LogRegr), '-', 100, '%')
print('тренировочная выборка =', len(x_train), '-', len(x_train)*100/len(LogRegr), '%')
print('тестова выборка =', len(x_test), '-', len(x_test)*100/len(LogRegr), '%')
print('размер dataset =', LogRegr.shape)

выборка всего датасет = 9742 - 100 %
тренировочная выборка = 6819 - 69.99589406692671 %
тестова выборка = 2923 - 30.00410593307329 %
размер dataset = (9742, 1447)


In [ ]:
# кол-во категорий +\- одинаковое, датасет сбалансирован
LogRegr['price_category'].value_counts()

high      3396
low       3178
medium    3168
Name: price_category, dtype: int64

In [ ]:
# обучим модель логист регрессии на тренировочной выборке на всех переменных
logreg = LogisticRegression(penalty='l2', C=1, max_iter=1000, fit_intercept=True, solver='liblinear')
logreg.fit(x_train, y_train)
# сделать предикт на тренировочных параметрах и выполнить оценку точности
pred_train = logreg.predict(x_train)
accuracy_score(y_train, pred_train)
# сделать предикт на тестовых параметрах и выполнить оценку точности
pred_test = logreg.predict(x_test)
accuracy_score(y_test, pred_test)
print('точность тренировочной выборки =', accuracy_score(y_train, pred_train))
print('точность тестовой выборки =', accuracy_score(y_test, pred_test))

точность тренировочной выборки = 0.7533362663147088
точность тестовой выборки = 0.7437564146424905


In [ ]:
# в логистической регрессии чаще всего можно получить хорошие результаты, настрив
# параметры регуляризации. она позволяет штрафовать модель за слишком высокие коэфициенты
# это помогает сделать модель более общей и не затачивать модель под тренировочную выборку
# за регуляризацию отвечает параметр penalty, он позволяет попробовать разные методы
# и параметр С, который настраивает мощность регуляризации

print(logreg.get_params())

{'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [ ]:
cross_val_score(logreg, x, y, cv=6)

array([0.7364532 , 0.75184729, 0.75307882, 0.76293103, 0.71842267,
       0.73444239])

# **Выводы о переобучении/недообучении модели на основании метрик ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ**


---

Модель слабо обучилась хорошо на тренировочной выборке и на тестовой выборке. 
Метрики качества в целом имеют одно значение точности от итерации к итерации кросс-валидации, те модель не склонна к переобучению.
Модель показывает стабильно высокие результаты на тестовой и тренировочной выборке, поэтому её не рекомендуется использовать в промышленном решении.


---




# **ДЕРЕВО РЕШЕНИЙ**
SKLEARN

---


ПРО ДЕРЕВЬЯ ИНТЕРЕСНАЯ ИНФО
ДЕРЕВО - СПЕЦИАЛЬНАЯ СТРУКТУРА ДАННЫХ, КОТОРАЯ СОСТОИТ ИЗ 2Х ЭЛЕМЕНТОВ
УЗЕЛ (NODE) - РЕШАЮЩИЕ ПРАВИЛО. ПЕРВЫЙ УЗЕЛ КОРНЕВОЙ
И ЛИСТ (LEAF)

КАК ФОРМИРУЕТСЯ ДЕРЕВО РЕШЕНИЙ ВО ВРЕМЯ ОБУЧЕНИЙ?


---
УЗЛЫ СОЗДАЮТСЯ ТАКИМ, ОБРАЗОМ, ЧТОБЫ МАКСИМАЛЬНО РАЗБИТЬ ВЫБОРКУ НА ДВА КЛАССА

ПРОВЕРЯЕТСЯ ACCURACY НА TRAIN (КАК МОДЕЛЬ ПРЕДСКАЗЫВАЕТ ДАННЫЕ, КОТОРЫЕ ОНА УЖЕ ВИДЕЛА) & TEST (МОДЕЛЬ ДАННЫЕ НЕ ВИДЕЛА, BLIND TEST)



# **МОДЕЛЬ DECISION TREE CLASSIFIER**

In [ ]:
# Удалить из dataset переменную price и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.

df_prepared = df3.drop(columns=['std_scaled_price', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date',
                    'age_category', 'short_model', 'date', 'price', 'id'])

DTC = df_prepared.copy()
DTC.shape

x = DTC.drop(['price_category'], axis=1)
y = DTC['price_category']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
# Обучение модели дерева решений с зафиксированным random_state на тренировочной выборке.
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

# сделать предикт на тренировочных параметрах и выполнить оценку точности
pred_train = tree.predict(x_train)
accuracy_score(y_train, pred_train)
# сделать предикт на тестовых параметрах и выполнить оценку точности
pred_test = tree.predict(x_test)
accuracy_score(y_test, pred_test)
print(f"   Полученные оценки:")
print('точность тренировочной выборки =', accuracy_score(y_train, pred_train))
print('точность тестовой выборки =', accuracy_score(y_test, pred_test))
print('размер dataset =', DTC.shape)
print(cross_val_score(tree, x, y, cv=5))

   Полученные оценки:
точность тренировочной выборки = 1.0
точность тестовой выборки = 0.6948340745809101
размер dataset = (9742, 1446)
[0.72498717 0.69881991 0.70277207 0.70225873 0.70687885]


In [ ]:
 # выведем features и отсортируем по убыванию, чтобы понять, какие фичи вносят больший результат
tree.feature_importances_
f_imp_list = list(zip(x_train.columns, tree.feature_importances_))
f_imp_list.sort(key = lambda x: x[1], reverse=True)
f_imp_list
 # Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
columns_mask = (tree.feature_importances_ != 0)
x_train_clear = x_train[x_train.columns[columns_mask]]
x_test_clear = x_test[x_test.columns[columns_mask]]

In [ ]:

 # Заново обучите модель и измерьте качество.
x_train_clear, x_test_clear, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
# Обучение модели дерева решений с зафиксированным random_state на тренировочной выборке.
tree = DecisionTreeClassifier(max_depth=1500)
tree.fit(x_train_clear, y_train)

# сделать предикт на тренировочных параметрах и выполнить оценку точности
pred_train = tree.predict(x_train_clear)
accuracy_score(y_train, pred_train)
# сделать предикт на тестовых параметрах и выполнить оценку точности
pred_test = tree.predict(x_test_clear)
accuracy_score(y_test, pred_test)
print(f"   Полученные оценки:")
print('точность тренировочной выборки =', accuracy_score(y_train, pred_train))
print('точность тестовой выборки =', accuracy_score(y_test, pred_test))
print('размер dataset =', DTC.shape)
print(cross_val_score(tree, x, y, cv=5))

   Полученные оценки:
точность тренировочной выборки = 1.0
точность тестовой выборки = 0.6989394457748889
размер dataset = (9742, 1446)
[0.717804   0.69830682 0.69815195 0.70020534 0.72022587]


# **Выводы о переобучении/недообучении модели на основании метрик МОДЕЛИ DTC**


---

Модель хорошо обучилась на тренировочной выборке и на тестовой выборке. 
В целом результат метрики остаётся хорошим, и модель сохраняет свою предиктивную способность. 



---



# **Модель Random ForestClassifier (случайного леса)**

In [ ]:
# Удалить из dataset переменную price и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.

df_prepared = df3.drop(columns=['id', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date',
                    'age_category', 'short_model', 'date', 'price', 'std_scaled_price'])

RFC = df_prepared.copy()
RFC.shape

x = RFC.drop(['price_category'], axis=1)
y = RFC['price_category']
# Обучим классификации с помощью RandomForestClassifier
# На новых урезанных семплах тренировочной и тестовой выборок обучиме модель случайного леса 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
clf_rfc = RandomForestClassifier()
clf_rfc.fit(x_train, y_train)
# сделать предикт на тренировочных параметрах и выполнить оценку точности
pred_train = clf_rfc.predict(x_train)
accuracy_score(y_train, pred_train)
# сделать предикт на тестовых параметрах и выполнить оценку точности
pred_test = clf_rfc.predict(x_test)
accuracy_score(y_test, pred_test)
print(f"   Полученные оценки:")
print(f'на тренировочной выборке: {accuracy_score(y_train, pred_train)}')
print(f'на тестовой выборке: {accuracy_score(y_test, pred_test)}')
print(cross_val_score(clf_rfc, x, y, cv=5))

   Полученные оценки:
на тренировочной выборке: 1.0
на тестовой выборке: 0.7461512145056449
[0.75833761 0.76038994 0.74486653 0.75410678 0.75821355]


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
clf_rfc_update = RandomForestClassifier()
params = {
            "n_estimators": [30],
            "bootstrap": [False, True],
            "max_depth": range(20, 50),
            "max_features": ["auto", "sqrt", "log2"],
            "min_samples_leaf": [1, 2, 3],
            "min_samples_split": [2, 3]
         }

In [ ]:
grid = GridSearchCV(clf_rfc_update, params, cv=5)
1 + 1
grid.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [False, True], 'max_depth': range(20, 50),
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3], 'n_estimators': [30]})

In [ ]:
grid.best_params_

{'bootstrap': False,
 'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 30}

In [ ]:
clf_rfc_update = RandomForestClassifier(bootstrap=False, n_estimators=30, max_depth=40, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, random_state=50)
clf_rfc_update.fit(x_train, y_train)

# сделать предикт на тренировочных параметрах и выполнить оценку точности
pred_train = clf_rfc_update.predict(x_train)
accuracy_score(y_train, pred_train)
# сделать предикт на тестовых параметрах и выполнить оценку точности
pred_test = clf_rfc_update.predict(x_test)
accuracy_score(y_test, pred_test)
print(f"   Полученные оценки:")
print(f'на тренировочной выборке: {accuracy_score(y_train, pred_train)}')
print(f'на тестовой выборке: {accuracy_score(y_test, pred_test)}')
print(cross_val_score(clf_rfc_update, x, y, cv=5))

   Полученные оценки:
на тренировочной выборке: 1.0
на тестовой выборке: 0.7461512145056449
[0.74551052 0.75474602 0.74897331 0.76232033 0.75      ]


# **Выводы о переобучении/недообучении модели на основании метрик МОДЕЛИ RFC**


---

Модель хорошо обучилась на тренировочной выборке и на тестовой выборке. 
В целом результат метрики остаётся хорошим, и модель сохраняет свою предиктивную способность. 



---

# **Нейронная сеть**

In [ ]:
# Модель многослойный персептрон
# Подготовьте данные: удалите колонки, которые косвенно содержат информацию о целевой переменной (odometer/price_std, price_category).

df_prepared = df3.drop(columns=['id', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'title_status', 'transmission', 'image_url', 'description', 'state', 'posting_date',
                    'age_category', 'short_model', 'date', 'price', 'std_scaled_price'])

neuro = df_prepared.copy()
neuro.shape

x = neuro.drop(['price_category'], axis=1)
y = neuro.price_category
# инициализация модели по нейронным сетям
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print('выборка всего датасет =', len(neuro), '-', 100, '%')
print('тренировочная выборка =', len(x_train), '-', len(x_train)*100/len(neuro), '%')
print('тестова выборка =', len(x_test), '-', len(x_test)*100/len(neuro), '%')

выборка всего датасет = 9742 - 100 %
тренировочная выборка = 7793 - 79.99384110039006 %
тестова выборка = 1949 - 20.006158899609936 %


In [ ]:
#запустим на этих же данных многослойный пресиптрона из from sklearn.neural_network import MLPClassifier
# здесь есть задача как для классификатора так и для регрессии. 
# наша текущая задача - классификатор
# создадим два слоя, hidden_layer_sizes=(100,20), на первом оставим 100 нейронов, на втором 20 нейронов
mlp = MLPClassifier(max_iter=1000, random_state=42, hidden_layer_sizes=(500,200))
mlp.fit(x_train, y_train)
# обучим модель, посчитаем точность 
pred_test = mlp.predict(x_test)
pred_train = mlp.predict(x_train)
print('точность тестовой выборки =', accuracy_score(y_test, pred_test))
print('точность тренировочной выборки =', accuracy_score(y_train, pred_train))
print(cross_val_score(mlp, x, y, cv=5))


точность тестовой выборки = 0.5772190867111339
точность тренировочной выборки = 0.5951494931348646
[0.49974346 0.40482299 0.59240246 0.34907598 0.62063655]


# **Выводы о переобучении/недообучении модели на основании метрик нейронной сети**


---

Модель слабо обучилась на тренировочной выборке и на тестовой выборке. 
Метрики качества в целом имеют одно значение точности от итерации к итерации кросс-валидации, те модель не склонна к переобучению.
Модель показывает стабильно низкие результаты на тестовой и тренировочной выборке, поэтому её не рекомендуется использовать в промышленном решении.


---

# **РЕЗУЛЬТАТЫ**


---


По результатам моделирования наилучший результат показала модель случайного леса

# **Pickle**


---



In [ ]:
#cериализация модели с помощью пикл
filename = 'model.pickle'

with open(filename, 'wb') as file:
  pickle.dump(clf_rfc_update, file) # передаем объект, который нужно запиклить, те модель по случайному лесу

In [ ]:
with open(filename, 'rb') as file:
  model_from_pickle = pickle.load(file)

In [ ]:
model_from_pickle

RandomForestClassifier(bootstrap=False, max_depth=40, max_features='sqrt',
                       n_estimators=30, random_state=50)